In [1]:
from renormalizer.utils import Quantity
from renormalizer import Model, Mpo, Mps, optimize_mps
from renormalizer.model.basis import BasisSimpleElectron, BasisSHO, Op
import numpy as np

2023-03-07 15:52:10,592[INFO] Use NumPy as backend
2023-03-07 15:52:10,597[INFO] use 64 bits


In [2]:
# Hubbard-Holstein Model
L = 4
t = 1.
U = 2.
omega = 5.0
lamb = 3.6
g = np.sqrt(lamb * omega / 2)
nphonon = [3] * L

In [3]:
basis = []

# Electron Basis
for imol in range(2 * L):
    basis.append(BasisSimpleElectron(imol))

# Phonon Basis
for isite, iph in enumerate(range(L)):
    basis.append(BasisSHO((iph, 0), omega, nphonon[isite]))

In [4]:
ham = []

# electron hopping term

hop_term_index = np.array([[[i, i + 2], [i + 2, i]] for i in range(2 * (L - 1))]).reshape(-1,2)
for imol, jmol in hop_term_index:
    hop_term = Op(r"a^\dagger a", [imol, jmol], -t)
    ham.append(hop_term)

# electron interaction term
int_term_index = np.array([[2 * i, 2 * i, 2 * i + 1, 2 * i + 1] for i in range(L)]).reshape(-1,4)
for imol, jmol, kmol, lmol in int_term_index:
    n_up = Op(r"a^\dagger a", [imol, jmol], U)
    n_dn = Op(r"a^\dagger a", [kmol, lmol], 1.)
    int_term = n_up * n_dn
    ham.append(int_term)

# phonon energy term \omega b_i^\dagger b_i

for isite, iph in enumerate(range(L)):
    cre = Op(r"b^\dagger", (iph, 0), omega)
    des = Op(r"b", (iph, 0), 1.)
    ham.append(cre * des)


# electron-phonon coupling term
for isite, (imol, jmol, kmol, lmol) in enumerate(int_term_index):
    iph = range(L)[isite]
    n_up = Op(r"a^\dagger a", [imol, jmol], 1.)
    n_dn = Op(r"a^\dagger a", [kmol, lmol], 1.)
    cre = Op(r"b^\dagger", (iph, 0), g)
    des = Op(r"b", (iph, 0), g)
    ham.append(n_up * cre)
    ham.append(n_dn * cre)
    ham.append(n_up * des)
    ham.append(n_dn * des)

In [5]:
model = Model(basis, ham_terms=ham)
print(model.nsite)
mpo = Mpo(model)

M = 800
procedure = [[M, 0.4], [M, 0.2], [M, 0.1], [M, 0], [M, 0], [M,0], [M,0]]
mps = Mps.random(model, L, M)   # 4 is the electron number

2023-03-07 15:52:22,751[DEBUG] # of operator terms: 36
2023-03-07 15:52:22,753[DEBUG] symbolic mpo algorithm: Hopcroft-Karp
2023-03-07 15:52:22,759[DEBUG] Input operator terms: 36
2023-03-07 15:52:22,794[DEBUG] After combination of the same terms: 36


12


In [10]:
mps.optimize_config.procedure = procedure

import time
start = time.time()
energies, mps = optimize_mps(mps.copy(), mpo)
end = time.time()

2023-03-07 15:53:02,209[INFO] optimization method: 2site
2023-03-07 15:53:02,212[INFO] e_rtol: 1e-06
2023-03-07 15:53:02,217[INFO] e_atol: 1e-08
2023-03-07 15:53:02,222[INFO] procedure: [[800, 0.4], [800, 0.2], [800, 0.1], [800, 0], [800, 0], [800, 0], [800, 0]]
2023-03-07 15:53:02,265[DEBUG] isweep: 0
2023-03-07 15:53:02,267[DEBUG] mmax, percent: 800, 0.4
2023-03-07 15:53:02,269[DEBUG] mps current size: 215.5KiB, Matrix product bond dim:[1, 2, 4, 8, 16, 30, 50, 70, 70, 27, 9, 3, 1]
2023-03-07 15:53:02,275[DEBUG] optimize site: [10, 11]
2023-03-07 15:53:02,280[DEBUG] use direct eigensolver
2023-03-07 15:53:02,287[DEBUG] energy: -11.135300766600563
2023-03-07 15:53:02,300[DEBUG] optimize site: [9, 10]
2023-03-07 15:53:02,302[DEBUG] use direct eigensolver
2023-03-07 15:53:02,515[DEBUG] energy: -11.135300766600567
2023-03-07 15:53:02,518[DEBUG] optimize site: [8, 9]
2023-03-07 15:53:02,529[DEBUG] use davidson, HC hops: 1
2023-03-07 15:53:02,532[DEBUG] energy: -11.135300766600578
2023-03-0

In [11]:
print(end - start)
print(min(energies))

1.419983148574829
-11.135300766600606
